# Intro

Data scientists tend to focus on **prediction** because that's where conventional machine learning excels. But real world decision-making involves both prediction and **optimization**.  After predicting what will happen, you decide what to do about it.

Optimization gets less attention than it deserves. So this micro-challenge will test your optimization skills as you write a function to improve how airlines set prices.

![Imgur](https://i.imgur.com/AKrbLMR.jpg)


# The Problem

You recently started Aviato.com, a startup that helps airlines set ticket prices. 

Aviato's success will depend on a function called `pricing_function`.  This notebook already includes a very simple version of `pricing_function`.  You will modify `pricing_function` to maximize the total revenue collected for all flights in our simulated environment.

For each flight, `pricing_function` will be run once per (simulated) day to set that day's ticket price. The seats you don't sell today will be available to sell tomorrow, unless the flight leaves that day.

Your `pricing_function` is run for one flight at a time, and it takes following inputs:
- **Number of days until the flight**
- **Number of seats they have left to sell**
- **A variable called `demand_level` that determines how many tickets you can sell at any given price. **

The quantity you sell at any price is:
> quantity_sold = demand_level - price

Ticket quantities are capped at the number of seats available.

Your function will output the ticket price.

You learn the `demand_level` for each day at the time you need to make predictions for that day. For all days in the future, you only know `demand_level` will be drawn from the uniform distribution between 100 and 200.  So, for any day in the future, it is equally likely to be each value between 100 and 200.

In case this is still unclear, some relevant implementation code is shown below.

# The Simulator
We will run your pricing function in a simulator to test how well it performs on a range of flight situations.  **Run the following code cell to set up your simulation environment:**

In [1]:
import sys
sys.path.append('../input')
from flight_revenue_simulator import simulate_revenue, score_me


In case you want to check your understanding of the simulator logic, here is a simplified version of some of the key logic (leaving out the code that prints your progress). If you feel you understand the description above, you can skip reading this code.

```python
def _tickets_sold(p, demand_level, max_qty):
        quantity_demanded = floor(max(0, p - demand_level))
        return min(quantity_demanded, max_qty)

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, demand_level_min=100, demand_level_max=200):
    if (days_left == 0) or (tickets_left == 0):
        return rev_to_date
    else:
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        return _total_revenue(days_left = days_left-1, 
                              tickets_left = tickets_left-q, 
                              pricing_function = pricing_function, 
                              rev_to_date = rev_to_date + p * q,
                              demand_level_min = demand_level_min,
                              demand_level_max = demand_level_max
                             )
```

# Your Code

Here is starter code for the pricing function.  If you use this function, you will sell 10 tickets each day (until you run out of tickets).

In [2]:
def pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    price = demand_level - 10
    return price

To see a small example of how your code works, test it with the following function:

In [3]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 109. Price set to $99. Sold 10 tickets. Daily revenue is 993. Total revenue-to-date is 993. 40 seats remaining
6 days before flight: Started with 40 seats. Demand level: 178. Price set to $168. Sold 10 tickets. Daily revenue is 1677. Total revenue-to-date is 2670. 30 seats remaining
5 days before flight: Started with 30 seats. Demand level: 167. Price set to $157. Sold 10 tickets. Daily revenue is 1574. Total revenue-to-date is 4244. 20 seats remaining
4 days before flight: Started with 20 seats. Demand level: 123. Price set to $113. Sold 10 tickets. Daily revenue is 1132. Total revenue-to-date is 5376. 10 seats remaining
3 days before flight: Started with 10 seats. Demand level: 172. Price set to $162. Sold 10 tickets. Daily revenue is 1619. Total revenue-to-date is 6994. 0 seats remaining
This flight is booked full.
Total Revenue: $6994


6994.409838146537

You can try simulations for a variety of values.

Once you feel good about your pricing function, run it with the following cell to to see how it performs on a wider range of flights.

In [4]:
def pricing_function(days_left, tickets_left, demand_level):
    """My pricing function"""
    import math
    # precomputing useful values
    tickets_per_day = round(tickets_left / days_left)
    mean_demand = 150
    mean_price = mean_demand - tickets_per_day
    demand_mean_diff = abs(demand_level - mean_demand) / 50
    
    # Price for last day and far days
    if days_left == 1:
        return demand_level - tickets_left
    if days_left > 7 and tickets_per_day <= 10:
        if demand_level < 175:
            return demand_level
    
    # Price for last 7 days
    if demand_level < mean_demand:
        factor = math.sqrt(1 - demand_mean_diff)
    else:
        factor = (1 + demand_mean_diff) ** 2.5
    today_seats = factor * tickets_per_day
    price = demand_level - today_seats    
    return price

simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 104. Price set to $102. Sold 1 tickets. Daily revenue is 102. Total revenue-to-date is 102. 49 seats remaining
6 days before flight: Started with 49 seats. Demand level: 145. Price set to $138. Sold 7 tickets. Daily revenue is 964. Total revenue-to-date is 1066. 42 seats remaining
5 days before flight: Started with 42 seats. Demand level: 159. Price set to $147. Sold 11 tickets. Daily revenue is 1614. Total revenue-to-date is 2680. 31 seats remaining
4 days before flight: Started with 31 seats. Demand level: 170. Price set to $152. Sold 18 tickets. Daily revenue is 2729. Total revenue-to-date is 5408. 13 seats remaining
3 days before flight: Started with 13 seats. Demand level: 100. Price set to $100. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 5408. 13 seats remaining
2 days before flight: Started with 13 seats. Demand level: 141. Price set to $135. Sold 5 tickets. Daily revenue is 677. Total revenue-to-date i

7551.258595632313

In [5]:
score_me(pricing_function)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $18246
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $8325
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2771
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $439


<IPython.core.display.Javascript object>

Average revenue across all flights is $7445


# Discuss
Want to discuss your solution or hear what others have done?  There is a [discussion thread](https://www.kaggle.com/general/62469) just for you.

---
*This micro-challenge is from an exercise in an upcoming Optimization course on **[Kaggle Learn](https://www.kaggle.com/Learn?utm_medium=website&utm_source=kaggle.com&utm_campaign=micro+challenge+2018)**.  If you enjoyed this challenge and want to beef up your data science skills, you might enjoy our other courses.*